In [1]:
import spacy
nlp=spacy.load('en_core_web_sm')
def remove_stop_punct(s):
        from nltk.corpus import stopwords
        stopwords=stopwords.words('english')
        doc=nlp(s)
        #k=sorted([".","DT","TO","CC","IN"])
        #st=" ".join(t.text for t in doc if (t.pos_ not in ["PUNCT"] )and(t.text.lower() not in stopwords))
        st=" ".join(t.text for t in doc if (t.pos_ not in ["PUNCT"] ))
        return st.strip()

In [2]:
f=open('/home/judson/Desktop/sentenceSeg/POS_IntraSent/decision_POS.txt',"a")
f1=open('/home/judson/Desktop/sentenceSeg/Labelled/decision.txt',"r")
txt=f1.readlines()
for i in txt[374:]:
    s=remove_stop_punct(i)
    t=nlp(s)
    pos=[i.tag_ for i in t]
    f.writelines(s+"\t"+",".join(pos)+"\t<DECISION>\n")

TypeError: Argument 'string' has incorrect type (expected unicode, got str)

In [1]:
f1=open('/home/judson/Desktop/sentenceSeg/Labelled/arguments.txt',"r")
txt=f1.readlines()

In [5]:
import spacy
nlp=spacy.load('en_core_web_sm')
def remove_stop_punct(s):
        from nltk.corpus import stopwords
        stopwords=stopwords.words('english')
        doc=nlp(s)
        #k=sorted([".","DT","TO","CC","IN"])
        #st=" ".join(t.text for t in doc if (t.pos_ not in ["PUNCT"] )and(t.text.lower() not in stopwords))
        st=" ".join(t.text for t in doc if (t.pos_ not in ["PUNCT"] ))
        return st.strip()
#print remove_stop_punct(s)

In [4]:
def dict_conv(st):
    doc=nlp(st)
    root=[]
    for i in doc:
        if i.dep_=="ROOT":
            root.append(i.i)
    d={tok.i:[tok.text,tok.dep_,tok.head.text,tok.tag_,[child.i for child in tok.children]] for tok in doc}
    return [d,root]

In [3]:
import networkx
def dfs(sd, start, end = "$$"):   
    h=networkx.Graph(sd)
    ptr=[]
    for pth in networkx.all_simple_paths(h,start,"$$"):
        ptr.append(pth[:-1])
    return ptr

In [4]:
def parse_depend(dt,root):
    #md= {i:j[3] for i,j in dt.items()}
    #print dt
    sd=dict()
    sd["$$"]=[]
    for i,j in dt.items():
        if j[4]==[]:
            sd[i]=["$$"]
        else:
            sd[i]=[k for k in j[4]]
    #print sd
    #m=""
    #for i,j in dt.items():
    #    if j[0]=="ROOT":
    #        print j[0]
    #        m=str(i)
    #        break
    #print m
    #print sd
    ml=[]
    for i in root:
        ml.extend(dfs(sd,i,"$$"))
    #print ml
    mpos=[[dt[j][3] for j in i]for i in ml]
    mdep=[[dt[j][1] for j in i]for i in ml]
    return mdep,mpos

In [2]:
#input nested string list output string
def To_str(nlst):
    st="("
    for i in nlst:
        st=st+"("
        st=st+",".join(i)
        st=st+"),"
    return st[:-1]+")"

In [6]:
f=open('/home/judson/Desktop/sentenceSeg/Dep_IntraSent/Dep_decision',"a")
f1=open('/home/judson/Desktop/sentenceSeg/Labelled/decision.txt',"r")
decision_pickle=[]
txt=f1.readlines()
for i in txt:
    s=remove_stop_punct(i)
    #print s
    #t=nlp(s)
    dep,_=parse_depend(*dict_conv(s))
    decision_pickle.append(dep)
    st=To_str(dep)
    #f.writelines(s+"\t"+st+"\t<DECISION>\n")
    #print (s+"\t"+st+"\t<ARGUMENTS>\n")

In [16]:
f=open('/home/judson/Desktop/sentenceSeg/Dep_POS_IntraSent/dep_pos_decision.txt',"a")
f1=open('/home/judson/Desktop/sentenceSeg/Labelled/arguments.txt',"r")
txt=f1.readlines()
for i in txt:
    s=remove_stop_punct(i)
    #print s
    #t=nlp(s)
    dep,pos=parse_depend(*dict_conv(s))
    st=To_str(dep)
    st1=To_str(pos)
    f.writelines(s+"\t"+st+"\t"+st1+"\t<DECISION>\n")

In [7]:
#find unique nested list
#input ip_list->nested list
#output uniq_list->nested list
def uniq_list(ip_list):
    uniq_list=[ip_list[0]]
    for i in ip_list:
        k=0
        for j in uniq_list:
            if j==i:
                k=0
                break
            else:
                k=1
        if k==1:
            uniq_list.append(i)
    return uniq_list

In [9]:
import pickle
f=open("decision.pickle","wb")
pickle.dump(uniq_list(decision_pickle),f,protocol=2)

In [1]:
def mod_parse_depend(dt,root):
    #md= {i:j[3] for i,j in dt.items()}
    #print dt
    sd=dict()
    sd["$$"]=[]
    for i,j in dt.items():
        if j[4]==[]:
            sd[i]=["$$"]
        else:
            sd[i]=[k for k in j[4]]
    ml=[]
    for i in root:
        ml.extend(dfs(sd,i,"$$"))
    #print ml
    mpos=[[dt[j][3] for j in i]for i in ml]
    mdep=[[dt[j][1] for j in i]for i in ml]
    mname=[[dt[j][0] for j in i]for i in ml]
    return mdep,mpos,mname

In [6]:
p,d,n=mod_parse_depend(*dict_conv("Apple is looking at buying U.K.startup for $1 billion"))

In [32]:
def mod_file_write(p,d,n):
    mn=[[] for _ in range(len(p))]
    for i in range(len(p)):
        mn[i]=[(p[i][j],n[i][j])for j in range(len(p[i]))]
    st="["
    for i in mn:
        st=st+"["
        for j ,k in i:
            st=st+"("+j+","+k+"),"
        st=st[:-1]+"],"
    st=st[:-7]+"]]"
    return st+"\n"

In [38]:
f=open("/home/judson/Desktop/sentenceSeg/Labelled1/decision.txt","r")
#f1=open("/home/judson/LabelledDepparsed.txt","a")
id_r=f.readlines()
id_r=id_r[:4]+id_r[5:6]
f1.writelines("Decision:\n")
for i in id_r:
    f1.writelines(str(i+":"+mod_file_write(*mod_parse_depend(*dict_conv(i)))))